In [3]:
from pydantic import BaseModel

In [4]:
class User(BaseModel):
    first_name:str 
    last_name:str
    age:int

In [7]:
User(first_name="pranay",last_name="reddy",age=26)

User(first_name='pranay', last_name='reddy', age=26)

In [8]:
from pydantic import ValidationError

Validation Exceptions

In [12]:
try:
    User(first_name="pranay",last_name="reddy",age="junk")
except ValidationError as ex:
    exceptions=ex


In [14]:
exceptions.json() # to get exception in json format
exceptions.errors() # to get exceptions in dict format

[{'type': 'int_parsing',
  'loc': ('age',),
  'msg': 'Input should be a valid integer, unable to parse string as an integer',
  'input': 'junk',
  'url': 'https://errors.pydantic.dev/2.9/v/int_parsing'}]

Deserilizing Data

In [17]:
data={"first_name":"pranay","last_name":"reddy","age":42}

In [19]:
p=User.model_validate(data)
p

User(first_name='pranay', last_name='reddy', age=42)

In [21]:
data_json="""
{"first_name":"pranay",
"last_name":"reddy",
"age":43}"""

p=User.model_validate_json(data_json)
p

User(first_name='pranay', last_name='reddy', age=43)

Required vs optional params

In [22]:
try:
    User(age=42)
except ValidationError as e:
    print(e)

2 validation errors for User
first_name
  Field required [type=missing, input_value={'age': 42}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
last_name
  Field required [type=missing, input_value={'age': 42}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing


In [23]:
class User(BaseModel):
    first_name:str 
    last_name:str
    age:int = 0
    

In [24]:
User.model_fields

{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0)}

In [25]:
u=User(first_name="tharun",last_name="reddy")
u

User(first_name='tharun', last_name='reddy', age=0)

Nullable fields

In [30]:
class User(BaseModel):
    first_name:str|None =None
    last_name:str
    age: int = 20 

In [31]:
User.model_fields

{'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=20)}

In [32]:
p=User(last_name="reddy")

In [34]:
from typing import Union
class User(BaseModel):
    first_name:Union[str,None]=None
    last_name:str
    age:int=0

U=User(last_name="raja")
U


User(first_name=None, last_name='raja', age=0)

In [38]:
from typing import Optional

class User(BaseModel):
    first_name:Optional[str]
    last_name:str
    age:int=0

User.model_fields
#defualt is not passed so required is true for firstname

{'first_name': FieldInfo(annotation=Union[str, NoneType], required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0)}

In [44]:
class User(BaseModel):
    first_name:Optional[str]="tharun"
    last_name:str
    age:int=0
    lucky_nums:list[int]=[]

In [45]:
User.model_fields

{'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, default='tharun'),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0),
 'lucky_nums': FieldInfo(annotation=list[int], required=False, default=[])}

In [48]:
u=User(last_name="reddy",lucky_nums=[1,"2",3.0,4])
u

User(first_name='tharun', last_name='reddy', age=0, lucky_nums=[1, 2, 3, 4])

In [49]:
for i in u.lucky_nums:
    print(i)

1
2
3
4


Aliases and field classes

In [52]:
data = {"id":100,"first name":"pranay","last name":"smith","age in years":30}


In [54]:
from pydantic import Field
class User(BaseModel):
    id_:int=Field(alias="id")
    first_name:str=Field(alias="first name")
    last_name:str=Field(alias="last name")
    age:int=Field(alias=("age in years"))

u=User.model_validate(data)
u



User(id_=100, first_name='pranay', last_name='smith', age=30)

In [56]:
p=u.model_dump()
p

{'id_': 100, 'first_name': 'pranay', 'last_name': 'smith', 'age': 30}

In [59]:
p=u.model_dump_json()
p

'{"id_":100,"first_name":"pranay","last_name":"smith","age":30}'

In [60]:
p=u.model_dump(by_alias=True)
p

{'id': 100, 'first name': 'pranay', 'last name': 'smith', 'age in years': 30}

In [62]:
p=u.model_dump_json(by_alias=True)
p

'{"id":100,"first name":"pranay","last name":"smith","age in years":30}'

In [69]:
class User(BaseModel):
    first_name:str|None =Field(alias="FirstName",default=None)
    last_name :str = Field(alias="lastName")



data={"last_name":"smith"}

u=User.model_validate(data)
U
#to make it work with model field use below config

ValidationError: 1 validation error for User
lastName
  Field required [type=missing, input_value={'last_name': 'smith'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

In [71]:

data={"last_name":"smith"}
from pydantic import Field
class User(BaseModel):
    start_name:str=Field(alias="first_name", default="pranay")
    lastname:str=Field(alias="last_name")

u=User.model_validate(data)
u

User(start_name='pranay', lastname='smith')

User(start_name='pranay', lastname='smith')

Model Config: populat by name

In [77]:
from pydantic import ConfigDict
data={"last_name":"reddy"}
class User(BaseModel):
    model_config=ConfigDict(populate_by_name=True)
    firstname:str|None=Field(alias="first_name",default=None)
    lastname:str=Field(alias="last_name")

u=User.model_validate(data)
u


User(firstname=None, lastname='reddy')

Mutable Defaults

In [80]:
class Mymodel(BaseModel):
    numbers:list[int]=[]

m1=Mymodel()
m2=Mymodel()

In [81]:
m1.numbers.extend([1,2,3])
m1

Mymodel(numbers=[1, 2, 3])

In [83]:
m2.numbers

[]

Default Factories

In [84]:
from datetime import datetime, timezone

class Log(BaseModel):
    dt:datetime=Field(default_factory=lambda : datetime.now(timezone.utc))
    message:str

log1=Log(message="my messagee1")
log2=Log(message="my message 2")
log1
log2

Log(dt=datetime.datetime(2024, 10, 22, 15, 46, 32, 744257, tzinfo=datetime.timezone.utc), message='my message 2')

Custom serilaizers

In [4]:
from pydantic import BaseModel
class Model(BaseModel):
    num:float

m=Model(num=1.0)
m.model_dump()

{'num': 1.0}

In [7]:
m=Model(num=1/3)
m.model_dump()

{'num': 0.3333333333333333}

In [13]:
from datetime import datetime,timezone
class Model(BaseModel):
    dt:datetime

m=Model(dt=datetime.now(timezone.utc))
m.model_dump()

{'dt': datetime.datetime(2024, 10, 22, 18, 8, 47, 15562, tzinfo=datetime.timezone.utc)}

In [14]:
m.model_dump_json()

'{"dt":"2024-10-22T18:08:47.015562Z"}'

above datetime is in different format based on model_dump and model_dump_json where internally serilization is happening using python conversion,
below let's write custom serilaizer like above inbuild serializer for date time

In [22]:
from pydantic import field_serializer
class Model(BaseModel):
    number: float

    @field_serializer("number")
    def serialize_float(self,value):
        return round(value,2)

M=Model(number=1/3)
M.model_dump()


{'number': 0.33}

In [23]:
M.model_dump_json()

'{"number":0.33}'

In [40]:
from pydantic import field_serializer,Field
from datetime import datetime,timezone
class Dtmodel(BaseModel):
    id:int
    dt:datetime = Field(default_factory = lambda: datetime.now(timezone.utc))
    @field_serializer("dt",when_used="json-unless-none")
    def serilaize_datetime_to_json(self,value):
        return value.strftime("%Y/%M/-%D")

M=Dtmodel(id=1)
M.model_dump()    

{'id': 1,
 'dt': datetime.datetime(2024, 10, 22, 18, 37, 26, 984450, tzinfo=datetime.timezone.utc)}

In [38]:
M.model_dump_json()


'{"id":1,"dt":"2024/35/-10/22/24"}'

custom Validators: two types pre validation and post validations

#below post validations 

In [4]:
from pydantic import BaseModel,field_validator,Field
class Model(BaseModel):
    id:int = Field(gt=-100)
    
    @field_validator("id")
    @classmethod
    def validate_id(cls,value):
        return abs(value)

Model(id=10)

Model(id=10)

In [5]:
Model(id=-10)

Model(id=10)

In [8]:
Model(id=-300)

ValidationError: 1 validation error for Model
id
  Input should be greater than -100 [type=greater_than, input_value=-300, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/greater_than

In [9]:
Model(id="-10")

Model(id=10)

In [23]:
class Mylist(BaseModel):
    numbers: list[int]=[]

    @field_validator("numbers")
    @classmethod
    def numbers_validator(cls,value):
        if len(set(value)) != len(value):
            raise ValueError("elements_must_be unique")
        return value

Mylist(numbers=[1,2,3])

Mylist(numbers=[1, 2, 3])

In [26]:
try: 
    Model(numbers=["1",1,"2",3.0])
except ValidationError as e:
    print(e)

1 validation error for Model
id
  Field required [type=missing, input_value={'numbers': ['1', 1, '2', 3.0]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing


Nested Models

In [27]:
data ={
    "firstName":"Arthur",
    "lastName":"Clarke",
    "born":{
        "place":{"country":"india","city":"hyderabad"},
    "date":"3001-01-01"
    }
}

In [29]:
from datetime import date

class Place(BaseModel):
    country:str
    city:str

class Born(BaseModel):
    place:Place
    dt:date=Field(alias="date")
class Person(BaseModel):
    first_name:str|None=Field(alias="firstName",default=None)
    last_name:str=Field(alias="lastName")
    born: Born|None =None




In [37]:
p=Person.model_validate(data)

In [38]:
p.model_dump()

{'first_name': 'Arthur',
 'last_name': 'Clarke',
 'born': {'place': {'country': 'india', 'city': 'hyderabad'},
  'dt': datetime.date(3001, 1, 1)}}

In [39]:
p.model_dump_json()

'{"first_name":"Arthur","last_name":"Clarke","born":{"place":{"country":"india","city":"hyderabad"},"dt":"3001-01-01"}}'

In [42]:
from pprint import pprint

pprint(p.model_dump())

{'born': {'dt': datetime.date(3001, 1, 1),
          'place': {'city': 'hyderabad', 'country': 'india'}},
 'first_name': 'Arthur',
 'last_name': 'Clarke'}


In [45]:
pprint(p.model_dump_json(indent=2))

('{\n'
 '  "first_name": "Arthur",\n'
 '  "last_name": "Clarke",\n'
 '  "born": {\n'
 '    "place": {\n'
 '      "country": "india",\n'
 '      "city": "hyderabad"\n'
 '    },\n'
 '    "dt": "3001-01-01"\n'
 '  }\n'
 '}')
